### 복습 (데이터프레임)
1. 민원(콜센터) 질의응답_다산콜센터_일반행정 문의_Training.json 파일을 로드 
2. 고객질문(요청), 상담사답변 컬럼의 문자 정규화 (특문 제거, 공백제거 )
3. 고객질문에 대한 즉각적인 상담사의 답변이 있는 행들만 남기고 나머지는 제거 
4. 고객의 질문과 상담사의 답변을 하나의 행으로 결합 
5. 인덱스를 초기화하고 label 컬럼을 생성하여 1을 대입 

In [50]:
import re
import pandas as pd

In [51]:
# 데이터 로드 
df = pd.read_json("../data/민원(콜센터) 질의응답_다산콜센터_일반행정 문의_Training.json")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50336 entries, 0 to 50335
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   도메인        50336 non-null  object
 1   카테고리       50336 non-null  object
 2   대화셋일련번호    50336 non-null  object
 3   화자         50336 non-null  object
 4   문장번호       50336 non-null  int64 
 5   고객의도       50336 non-null  object
 6   상담사의도      50336 non-null  object
 7   QA         50336 non-null  object
 8   고객질문(요청)   50336 non-null  object
 9   상담사질문(요청)  50336 non-null  object
 10  고객답변       50336 non-null  object
 11  상담사답변      50336 non-null  object
 12  개체명        50336 non-null  object
 13  용어사전       50336 non-null  object
 14  지식베이스      50336 non-null  object
dtypes: int64(1), object(14)
memory usage: 5.8+ MB


In [52]:
df.head()

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,다산콜센터,일반행정 문의,B2240,고객,1,지방세납부,,Q,지방세를 내려면 어떻게 해야됩니까?,,,,지방세,지방세/세금,"지방세,세금"
1,다산콜센터,일반행정 문의,B2240,상담사,2,,지방세납부,A,,,,이용하시는 은행의 사이트에서 지방세 납부가 가능합니다.,"은행, 사이트, 지방세, 납부",은행/공공기관/ 지방세/세금,"사이트,세금"
2,다산콜센터,일반행정 문의,B2240,고객,3,지방세납부,,Q,은행 어플에서도 됩니까?,,,,"은행, 어플",은행/공공기관,"어플,공공기관"
3,다산콜센터,일반행정 문의,B2240,상담사,4,,지방세납부,Q,,어떤 은행을 이용하고 계십니까?,,,은행,은행/공공기관,"은행,공공기관"
4,다산콜센터,일반행정 문의,B2240,고객,5,지방세납부,,A,,,기업은행을 이용하고 있습니다.,,기업은행,기업은행/상호,"기업은행,상호"


In [53]:
df.columns

Index(['도메인', '카테고리', '대화셋일련번호', '화자', '문장번호', '고객의도', '상담사의도', 'QA',
       '고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변', '개체명 ', '용어사전', '지식베이스'],
      dtype='object')

In [54]:
# 컬럼의 이름에서 좌우의 공백을 제거 (strip() -> str 내장함수)
df.columns.map( lambda x : x.strip() )

Index(['도메인', '카테고리', '대화셋일련번호', '화자', '문장번호', '고객의도', '상담사의도', 'QA',
       '고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변', '개체명', '용어사전', '지식베이스'],
      dtype='object')

In [55]:
[ x.strip() for x in df.columns ]

['도메인',
 '카테고리',
 '대화셋일련번호',
 '화자',
 '문장번호',
 '고객의도',
 '상담사의도',
 'QA',
 '고객질문(요청)',
 '상담사질문(요청)',
 '고객답변',
 '상담사답변',
 '개체명',
 '용어사전',
 '지식베이스']

In [56]:
cols = []
for x in df.columns:
    cols.append(x.strip())

df.columns = cols

In [57]:
# 문자의 정규화 함수 생성 
def nomarlize(text):
    text = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [58]:
# 2개의 컬럼에서 문자의 정규화 함수를 호출하여 데이터를 변환 
# case1 -> 각각 컬럼의 value들을 정규화함수에 대입하여 변환 (2번)
df2 = df.copy()
# 각각의 스리즈 데이터를 map() 함수를 이용
df2['고객질문(요청)'] = df2['고객질문(요청)'].map(nomarlize)
df2['상담사답변'] = df['상담사답변'].map(nomarlize)

In [59]:
# 데이터프레임에서 각각의 value들을 정규화 함수 대입 
# 구버전에서는 데이터프레임에서 map() 함수 사용 불가 
# df[['고객질문(요청)', '상담사답변']].map(nomarlize)
df[['고객질문(요청)', '상담사답변']] = df[['고객질문(요청)', '상담사답변']].applymap(nomarlize)

C:\Users\ekfla\AppData\Local\Temp\ipykernel_14136\4149374534.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['고객질문(요청)', '상담사답변']] = df[['고객질문(요청)', '상담사답변']].applymap(nomarlize)


In [60]:
# 고객질문(요청)컬럼의 이름을 고객질문으로 변경 
df.rename(columns={
    '고객질문(요청)' : '고객질문'
}, inplace=True )

In [61]:
# 고객의 질문에 대해서 상담사가 즉각적인 답변을 한 행들만 뽑아준다. 
# 조건 -> 고객의 질문이 존재하고 다음 행에 상담사의 답변이 존재한다. 
# (즉각적인 답변에 대한 고객질문의 조건식)
flag1 = (df['고객질문'] != '') & (df['상담사답변'].shift(-1) != "")
# 조건 -> 상담사답변이 존재하고 전 행의 고객질문이 존재하는경우 
# (즉각적인 답변에 대한 상담사답변의 조건식)
flag2 = (df['상담사답변'] != "") & (df['고객질문'].shift(1) != "")

In [62]:
# 즉각적인 답변에 대한 고객 질문만 모아둔 데이터프레임
q_df = df.loc[flag1, ]

In [63]:
# 즉각적인 답변인 상담사의 답변을 리스트로 생성
q_df['상담사답변'] = df.loc[flag2, '상담사답변'].to_list()

C:\Users\ekfla\AppData\Local\Temp\ipykernel_14136\114213427.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q_df['상담사답변'] = df.loc[flag2, '상담사답변'].to_list()


In [64]:
q_df.head(10)

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문,상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,다산콜센터,일반행정 문의,B2240,고객,1,지방세납부,,Q,지방세를 내려면 어떻게 해야됩니까,,,이용하시는 은행의 사이트에서 지방세 납부가 가능합니다.,지방세,지방세/세금,"지방세,세금"
6,다산콜센터,일반행정 문의,B2240,고객,7,지방세납부,,Q,은행을 직접방문해도 됩니까,,,방문납부도 가능합니다.,은행,은행/공공기관,"은행,공공기관"
12,다산콜센터,일반행정 문의,B2240,고객,13,지방세납부,,Q,버스로 가는 방법도 있습니까,,,도보로 가시는게 빠를 것 같습니다.,버스,버스/교통수단,"버스,교통수단"
14,다산콜센터,일반행정 문의,B2240,고객,15,지방세납부,,Q,다른 납부방법도 있습니까,,,위택스 사이트에서 납부하실수 있습니다.,,납부방법/지방세,
16,다산콜센터,일반행정 문의,B2240,고객,17,지방세납부,,Q,사이트 주소가 어떻게 됩니까,,,www.wetax.go.kr 입니다.,사이트,납부방법/지방세/ 위텍스/ 납부,"사이트,납부"
18,다산콜센터,일반행정 문의,B2240,고객,19,지방세납부,,Q,다른곳에서는 납부할수 없습니까,,,이용하시는 은행의 사이트에서도 지방세 납부가 가능합니다.,,납부방법/지방세/ 위텍스/ 납부,
20,다산콜센터,일반행정 문의,B2241,고객,1,지방세납부,,Q,지방세는 조회할 수 있습니까,,,간단한 본인확인 후 안내해드리겠습니다.,"지방세, 조회",지방세/세금,"조회,세금"
34,다산콜센터,일반행정 문의,B2241,고객,15,지방세납부,,Q,어떻게 납부합니까,,,온라인으로는 위택스나 은행 홈페이지에서 가능합니다.,납부,,납부
36,다산콜센터,일반행정 문의,B2241,고객,17,지방세납부,,Q,사이트 주소가 어떻게 됩니까,,,OOOO.OOO.go.kr 입니다.,사이트,사이트/은행/공공기관,"사이트,공공기관"
40,다산콜센터,일반행정 문의,B2242,고객,1,행사문의,,Q,서울시주최 페스티벌 예매해놨는데 예정대로 진행됩니까,,,현재로썬 진행될 예정입니다.,"페스티벌,예매",페스티벌/축제/ 예매/매입,"예매,매입"


In [65]:
# 인덱스를 초기화
q_df.reset_index(drop=True, inplace=True)

In [66]:
q_df = q_df[['고객질문', '상담사답변']]

In [67]:
# label 컬럼을 추가 하고 값은 1로 채워준다. 
q_df['label'] = 1

In [68]:
# label -> 1을 채운 이유 -> 해당하는 행의 질문과 답변은 정상적인 답변입니다. 
# label -> 0인 데이터들을 생성 -> 
# 기존의 질문과 답변에서 질문은 그대로 유지한채 답변은 원래의 답변을 제외한 다른 랜덤한 답으로 채운다. 
# 질문과 답변을 리스트로 생성 
answers_list = q_df[['고객질문', '상담사답변']].values.tolist()
answers_list

[['지방세를 내려면 어떻게 해야됩니까', '이용하시는 은행의 사이트에서 지방세 납부가 가능합니다.'],
 ['은행을 직접방문해도 됩니까', '방문납부도 가능합니다.'],
 ['버스로 가는 방법도 있습니까', '도보로 가시는게 빠를 것 같습니다.'],
 ['다른 납부방법도 있습니까', '위택스 사이트에서 납부하실수 있습니다.'],
 ['사이트 주소가 어떻게 됩니까', 'www.wetax.go.kr 입니다.'],
 ['다른곳에서는 납부할수 없습니까', '이용하시는 은행의 사이트에서도 지방세 납부가 가능합니다.'],
 ['지방세는 조회할 수 있습니까', '간단한 본인확인 후 안내해드리겠습니다.'],
 ['어떻게 납부합니까', '온라인으로는 위택스나 은행 홈페이지에서 가능합니다.'],
 ['사이트 주소가 어떻게 됩니까', 'OOOO.OOO.go.kr 입니다.'],
 ['서울시주최 페스티벌 예매해놨는데 예정대로 진행됩니까', '현재로썬 진행될 예정입니다.'],
 ['코로나로 다른 축제들은 취소됐는데 이건 취소 안됩니까', '네 철저한 방역수칙하에 진행할 예정입니다.'],
 ['환불할수 있습니까', '예매사이트 통해서 환불하실 수 있습니다.'],
 ['환불수수료 있습니까', '공연 일주일 전까진 10 수수료가 있습니다.'],
 ['코로나때문에 못가는건데도 수수료 내야합니까', '네 규정상 어쩔수 없습니다.'],
 ['참석했다가 코로나 걸리면 보상해줍니까', '철저한 방역수칙으로 진행하기에 걱정 안하셔도 됩니다.'],
 ['그래도 걸리면 보상해줍니까', '그 부분에 대해서는 개인이 조심을 하셔야기때문에 서울시에서는 따로 보상을 해드리진 않습니다.'],
 ['확산세가 더 강해지면 취소될 가능성 있습니까', '확산세가 심하면 취소 될 수도 있습니다.'],
 ['그럼 전액 환불되는겁니까', '네 맞습니다.'],
 ['청년저축계좌 지금 신청할 수 있습니까', '죄송하지만 이미 신청기간이 지났습니다.'],
 ['그럼 다른 정책은 없습니까', '희망두배 청년통장은 신청하실수 있습니

In [69]:
import numpy as np

In [70]:
# 질문은 유지한채 답변을 다른 답변으로 변경하여 새로운 2차원 리스트 생성 
neg_list = []
for q, a in answers_list:
    # q -> 질문
    # a -> 답변
    # 반복문에서 정상적인 답변을 제외한 나머지 답변의 리스트 생성 
    cand = [ a2 for q2, a2 in answers_list if a2 != a ]
    # 틀린 답변 하나를 선택 
    neg_a = np.random.choice(cand)
    neg_list.append([q, neg_a] )

In [71]:
answers_list2 = [ 
    ['A', 'a'] , 
    ['B', 'b'], 
    ['C', 'c'], 
    ['D', 'd'], 
    ['E', 'e']
]
neg_list2 = []
for q, a in answers_list2:
    # q -> 질문
    # a -> 답변
    # 1회차 반복 -> q('A'), a('a')
    # 반복문에서 정상적인 답변을 제외한 나머지 답변의 리스트 생성 
    # cand = [ a2 for q2, a2 in answers_list2 if a2 != a ]
    cand = []
    for q2, a2 in answers_list2:
        # a -> 'a'
        # a2 -> 반복 실행 'a', 'b', 'c', 'd', 'e'
        if a2 != a:
            cand.append(a2)
    # 틀린 답변 하나를 선택 
    neg_a = np.random.choice(cand)
    neg_list2.append([q, neg_a] )

In [72]:
neg_list2

[['A', np.str_('c')],
 ['B', np.str_('e')],
 ['C', np.str_('e')],
 ['D', np.str_('e')],
 ['E', np.str_('a')]]

In [73]:
neg_df = pd.DataFrame(neg_list, columns = ['고객질문', '상담사답변'])
neg_df.head()

,고객질문,상담사답변
0,지방세를 내려면 어떻게 해야됩니까,해당홈페이지 SH LH 에서 확인 가능 하십니다
1,은행을 직접방문해도 됩니까,서울시 전역의 공영주차장 견인차량 보관소 위치정도 확인 가능합니다.
2,버스로 가는 방법도 있습니까,가나아트 컬렉션 기획상설전 되겠습니다.
3,다른 납부방법도 있습니까,승인이 나면 고용노동부에 방문하셔서 교육을 들으셔야 합니다.
4,사이트 주소가 어떻게 됩니까,지방세를 부과 또는 징수에 이의가 있을 경우 감사원장을 통하여 심사청구 받는 제도되...


In [74]:
q_df.head()

,고객질문,상담사답변,label
0,지방세를 내려면 어떻게 해야됩니까,이용하시는 은행의 사이트에서 지방세 납부가 가능합니다.,1
1,은행을 직접방문해도 됩니까,방문납부도 가능합니다.,1
2,버스로 가는 방법도 있습니까,도보로 가시는게 빠를 것 같습니다.,1
3,다른 납부방법도 있습니까,위택스 사이트에서 납부하실수 있습니다.,1
4,사이트 주소가 어떻게 됩니까,www.wetax.go.kr 입니다.,1


In [75]:
neg_df['label'] = 0

In [76]:
# q_df와 neg_df를 단순한 행 결합 
dataset_df = pd.concat([q_df.head(1000), neg_df.head(1000)], axis=0, ignore_index=True)

In [77]:
dataset_df['label'].value_counts()

label
1    1000
0    1000
Name: count, dtype: int64

In [78]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

In [79]:
# train, test 셋으로 데이터프레임을 분할 
train_df, test_df = train_test_split(
    dataset_df, test_size=0.2, random_state=42, stratify=dataset_df['label']
)
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True))
ds = DatasetDict(
    {
        'train' : train_ds, 
        'validation' : test_ds
    }
)
ds

DatasetDict({
    train: Dataset({
        features: ['고객질문', '상담사답변', 'label'],
        num_rows: 1600
    })
    validation: Dataset({
        features: ['고객질문', '상담사답변', 'label'],
        num_rows: 400
    })
})

In [80]:
import torch
from transformers import AutoTokenizer, DataCollatorWithPadding
# BertForSequenceClassification : BertModel -> <CLS> 토큰 vector 추출 -> dropout -> linear 
from transformers import Trainer, TrainingArguments, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score


In [ ]:
# 토큰화 
MODEL_NAME = 'skt/kobert-base-v1'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
max_len = 256    # 장문의 데이터인 경우네는 512까지 조절 

def tok_fn(batch):
    # token화 할 데이터가 2개의 컬럼에 나눠져있다. 
    enc = tokenizer(
        batch['고객질문'], 
        batch['상담사답변'], 
        truncation = True, 
        max_length = max_len
    )
    # 일반적인 kobert 모델에서는 token_type_ids의 데이터 영역 필요x
    enc.pop("token_type_ids", None)   # 완전히 제거 
    return enc

# remove_columns -> 특정 컬럼의 데이터는 제외 -> label컬럼을 제외한 나머지 모두
tok_ds = ds.map(tok_fn, batched=True, 
                remove_columns=[col for col in dataset_df.columns\
                                 if col not in ['label']], 
                # 캐시 사용 안 함 
                load_from_cache_file= False, 
                desc = 'tokenizer clean'
                )

tokenizer clean: 100%|██████████| 400/400 [00:00<00:00, 12730.17 examples/s]


In [82]:
tok_ds

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1600
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 400
    })
})

In [83]:
# 배치마다 동적으로 padding토큰을 추가 
collator = DataCollatorWithPadding(
    tokenizer= tokenizer
)

In [85]:
# BertModel -> CLS -> dropout -> linear 
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, 
                                                      num_labels = 2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [86]:
# 평가 지표 함수 
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis= -1)
    return {
        'accuracy_score' : accuracy_score(labels, preds), 
        'f1_score' : f1_score(labels, preds) 
    }

In [87]:
args = TrainingArguments(
    output_dir="./kobert_pair_cls", 
    eval_strategy= 'epoch', 
    save_strategy= 'epoch', 
    logging_steps= 50, 
    learning_rate=5e-5, 
    weight_decay=0.01, 
    warmup_ratio=0.1, 
    num_train_epochs=3, 
    load_best_model_at_end=True, 
    metric_for_best_model='f1', 
    greater_is_better=True, 
    report_to=[]
)